In [46]:
import numpy as np
from prospect.models.templates import TemplateLibrary
from prospect.models.transforms import tburst_from_fage
from astropy.cosmology import Planck13

from argparse import Namespace
from scipy.special import gamma, gammainc

In [34]:
zreds = np.linspace(0,2,5)
# for z in zreds:
#     print(Planck13.lookback_time(z))
times = np.array([Planck13.lookback_time(z).value for z in zreds])
times

array([ 0.        ,  5.19203003,  7.93312573,  9.51795323, 10.51184098])

In [36]:
def parametric_sfr(times=None, tavg=1e-3, tage=1, **sfh):
    """Return the SFR (Msun/yr) for the given parameters of a parametric SFH,
    optionally averaging over some timescale.

    :param times: (optional, ndarray)
        If given, a set of ***lookback*** times where you want to calculate the sfr,
        same units as `tau` and `tage`

    :param tavg: (optional, float, default: 1e-3)
        If non-zero, average the SFR over the last `tavg` Gyr. This can help
        capture bursts.  If zero, the instantaneous SFR will be returned.

    :param sfh: optional keywords
        FSPS parametric SFH parametrs, e.g. sfh, tage, tau, sf_trunc

    :returns sfr:
        SFR in M_sun/year either for the lookback times given by `times` or at
        lookback time 0 if no times are given.  The SFR will either be
        instaneous or averaged over the last `tavg` Gyr.
    """
    if times is None:
        times = np.array(tage)

    pset = parametric_pset(tage=tage, **sfh)
    sfr, mass = compute_mass_formed(tage - times, pset)
    if tavg > 0:
        _, meps = compute_mass_formed((tage - times) - tavg, pset)
        sfr = (mass - meps) / (tavg * 1e9)
    return sfr

In [40]:
model_params = TemplateLibrary['parametric_sfh']
model_params.update(TemplateLibrary['burst_sfh'])
model_params.keys()

dict_keys(['zred', 'mass', 'logzsol', 'dust2', 'sfh', 'tage', 'imf_type', 'dust_type', 'tau', 'tburst', 'fburst', 'fage_burst'])

In [66]:
run_params = {}
run_params['zred'] = 3.548
run_params['mass'] = 1e8
run_params['logzsol'] = 0.0 # Solar metallicity
run_params['dust2'] = 0.0
run_params['sfh'] = 1 # tau model plus a constant component and a burst
run_params['tage'] = 0.3 # The age of the host (galaxy) in Gyrs
run_params['imf_type'] = 2 # (Kroupa 2001)
run_params['tau'] = 1.0 # The e-folding time for the SFH, range is 1e-1 to 1e2
# run_params['tburst']
run_params['fburst'] = 0.7 # The fraction of mass formed in an instantaneous burst of star formation
run_params['fage_burst'] = 0.25 # The fraction of the host age at which the burst occurred.
run_params['tburst'] = tburst_from_fage(**run_params) # The age of the universe (age of the host) when the burst occurred

run_params['const'] =  1.0 - run_params['fburst']

# run_params['sf_start'] = 0.0 # Start time of the SFH in Gyrs
# run_params['sf_trunc'] = 0.0 # Trunctation time of the SFH in Gyrs. 0.0 => no trunctation
# run_params['sf_slope'] # For sfh=5, this is the slope of the SFR *after* time sf_trunc

run_params

print(parametric_sfr(**run_params))

Namespace(mass=array([1.e+08]), sfh=1, sf_start=array([0]), tage=array([0.3]), tau=array([1.]), const=array([0.3]), fburst=array([0.7]), tburst=array([0.075]), sf_trunc=array([0]), sf_slope=array([0.]))


In [9]:
def compute_mass_formed(times, pset):
    """Compute the SFR and stellar mass formed in a parametric SFH,
    as a function of (forward) time.

    The linear portion of the Simha SFH (sfh=5) is defined as:
        psi(t) = psi_trunc + psi_trunc * sf_slope * (t - sf_trunc)
    where psi_trunc is the SFR of the delay-tau SFH at time sf_trunc

    :param times: ndarray of shape (nt,)
        Forward time in Gyr. Use times = pset.tage - t_lookback to
        convert from lookback times

    :param pset: Namespace instance
        The FSPS SFH parameters, assumed to be scalar or 1 element 1-d arrays.
        Usually the output of parametric_pset()

    :returns sfr: ndarray of shape (nt,)
        The instaneous SFR in M_sun/yr at each of `times`

    :returns mfromed: ndarray of shape (nt,)
        The total stellar mass formed from t=0 to `times`, in unist of M_sun
    """
    # TODO: use broadcasting to deal with multiple sfhs?

    # subtract sf_start
    tmass = pset.tage - pset.sf_start  # the age at which the mass is *specified*
    tprime = times - pset.sf_start     # the ages at which sfr and formed mass are requested

    if pset.sfh == 3:
        raise NotImplementedError("This method does not support tabular SFH")

    if np.any(tmass < 0):
        raise ValueError("SF never started (tage - sf_start < 0) for at least one input")

    if (pset.const + pset.fburst) > 1:
        raise ValueError("Constant and burst fractions combine to be > 1")

    if (pset.sfh == 0):
        # SSPs
        mfrac = 1.0 * (tprime > tmass)
        sfr = np.zeros_like(tprime)  # actually the sfr is infinity

    elif pset.sfh > 0:
        # Compute tau model component, for SFH=1,4,5
        #
        # Integration limits are from 0 to tmax and 0 to tprime, where
        #   - tmass is the tage, and
        #   - tprime is the given `time`,
        #   - ttrunc is where the delay-tau truncates
        ttrunc, tend = np.max(tprime), tmass
        if (pset.sf_trunc > 0) and (pset.sf_trunc > pset.sf_start):
            # Otherwise we integrate tau model to sf_trunc - sf_start
            ttrunc = pset.sf_trunc - pset.sf_start
            tend = min(tmass, ttrunc)

        # Now integrate to get mass formed by Tprime and by Tmax, dealing with
        # truncation that happens after sf_start but before Tmax and/or Tprime.
        power = 1 + int(pset.sfh > 3)
        total_mass_tau = pset.tau * gammainc(power, tend / pset.tau)

        tt = np.clip(tprime, 0, ttrunc)
        mass_tau = (tprime > 0.) * pset.tau * gammainc(power, tt / pset.tau)
        # The SFR at Tprime (unnormalized)
        sfr_tau = (tprime > 0.) * (tprime <= ttrunc) * (tprime / pset.tau)**(power-1.) * np.exp(-tprime / pset.tau)
        # fraction of tau component mass formed by tprime
        mfrac_tau = mass_tau / total_mass_tau

    # Add the constant and burst portions, for SFH=1,4.
    if ((pset.sfh == 1) or (pset.sfh == 4)):
        # Fraction of the burst mass formed by Tprime
        tburst = (pset.tburst - pset.sf_start)
        mfrac_burst = 1.0 * (tprime > tburst)
        # SFR from constant portion at Tprime (integrates to 1 at tmax)
        sfr_const = (tprime > 0) * 1.0 / tmass
        # fraction of constant mass formed by tprime
        mfrac_const = np.clip(tprime, 0, ttrunc) * sfr_const

        # Add formed mass fractions for each component, weighted by component fractions.
        # Fraction of the constant mass formed by Tprime is just Tprime/Tmax
        # TODO : The FSPS source does not include the tburst < tmass logic....
        mfrac = ((1. - pset.const - pset.fburst * (tburst < tmass)) * mfrac_tau +
                 pset.const * mfrac_const +
                 pset.fburst * mfrac_burst)

        # N.B. for Tprime = tburst, sfr is infinite, but we ignore that case.
        sfr = ((1. - pset.const - pset.fburst) * sfr_tau / total_mass_tau + pset.const * sfr_const)
        # We've truncated
        sfr *= (tprime <= ttrunc)

    # Add the linear portion, for Simha, SFH=5.
    # This is the integral of sfr_trunc*(1 - m * (T - Ttrunc)) from Ttrunc to Tz
    elif (pset.sfh == 5):
        #raise NotImplementedError

        m = -pset.sf_slope
        if (m > 0):
            # find time at which SFR=0, if m>0
            Tz = ttrunc + 1.0 / m
        else:
            # m <= 0 will never reach SFR=0
            Tz = np.max(tprime)

        # Logic for Linear portion
        if (ttrunc < 0):
            # Truncation does not occur during the SFH.
            total_mass_linear = 0.
            mass_linear = 0.
            sfr = sfr_tau / total_mass_tau
        else:
            # Truncation does occur, integrate linear to zero crossing or tage.
            Thi = min(Tz, tmass)
            sfr_trunc = (ttrunc/pset.tau) * np.exp(-ttrunc / pset.tau)
            total_mass_linear = (Thi > ttrunc) * sfr_trunc * linear_mass(Thi, ttrunc, m)
            mass_linear = (tprime > ttrunc) * sfr_trunc * linear_mass(tprime, ttrunc, m)
            mass_linear[tprime > Tz] = sfr_trunc * linear_mass(Tz, ttrunc, m)
            # SFR in linear portion
            sfr = sfr_trunc * (1 - m * (tprime - ttrunc)) / (total_mass_tau + total_mass_linear)
            sfr *= ((tprime > ttrunc) & (tprime <= Tz))
            # add portion for tau
            sfr[tprime <= ttrunc] = sfr_tau[tprime <= ttrunc] / (total_mass_tau + total_mass_linear)

        mfrac = (mass_tau + mass_linear) / (total_mass_tau + total_mass_linear)

    return pset.mass * sfr/1e9, pset.mass * mfrac

In [41]:
def tburst_from_fage(tage=0.0, fage_burst=0.0, **extras):
    """This function transfroms from a fractional age of a burst to an absolute
    age.  With this transformation one can sample in ``fage_burst`` without
    worry about the case ``tburst`` > ``tage``.

    Parameters
    ----------
    tage : float, Gyr
        The age of the host galaxy.

    fage_burst : float between 0 and 1
        The fraction of the host age at which the burst occurred.

    Returns
    -------
    tburst : float, Gyr
        The age of the host when the burst occurred (i.e. the FSPS ``tburst``
        parameter)
    """
    return tage * fage_burst

In [45]:
tburst_from_fage(tage=12.0, fage_burst=.2)

2.4000000000000004

In [8]:
def parametric_pset(logmass=None, **sfh):
    """Convert a dicionary of FSPS parametric SFH parameters into a
    namespace, making sure they are at least 1d vectors

    :param sfh: dicionary
        FSPS parameteric SFH parameters

    :returns pset:
        A Namespace instance with attributes giving the SFH parameters
    """
    # TODO: make multiple psets if any of the SFH parameters have np.size() > 1

    vectors = ["mass", "sf_start", "tage", "tau", "const", "fburst", "tburst", "sf_trunc", "sf_slope"]
    pset = Namespace(mass=1.0, sfh=4., sf_start=0, tage=1,
                     tau=1.,
                     const=0.,
                     fburst=0., tburst=1.,
                     sf_trunc=0, sf_slope=0.)

    if logmass:
        sfh["mass"] = 10**logmass
    for k in vars(pset).keys():
        if k in sfh:
            setattr(pset, k, sfh[k])
    # vectorize
    for k in vectors:
        setattr(pset, k, np.atleast_1d(getattr(pset, k)))

    return pset